In [8]:
import psycopg2

In [9]:
DATABASE_URI = {
    "dbname": "ae_takehome",
    "user": "postgres",
    "password": "your_password",
    "host": "localhost",
    "port": "5432"
}

In [10]:
# SQL script
sql_script = """
-- Create sales_performance table ----------------------------------------------
CREATE TABLE IF NOT EXISTS sales_performance (
    order_id INT PRIMARY KEY,
    customer_name VARCHAR(255),
    customer_id INT NOT NULL,
    region VARCHAR(100),
    segment VARCHAR(100),
    product_id INT NOT NULL,
    category VARCHAR(100),
    order_month INT NOT NULL,
    total_revenue NUMERIC(10, 2)
);

INSERT INTO sales_performance (order_id, customer_name, customer_id, region, segment, 
            product_id, category, order_month, total_revenue)
SELECT 
    f.order_id,
    f.customer_name,
    f.customer_id,
    c.region,    
    c.segment,
    f.product_id,
    p.category,
    EXTRACT(MONTH FROM f.order_date) AS order_month,
    f.total_revenue
FROM fact_sales f
LEFT JOIN dim_customer c ON f.customer_id = c.customer_id
LEFT JOIN dim_product p ON f.product_id = p.product_id;

-- Aggregated sales metrics by month, customer, and product -------------------
-- CALCULATE MONTHLY REVENUE
SELECT
    order_month,
    SUM(total_revenue) AS monthly_revenue
FROM
    sales_performance
GROUP BY
    order_month
ORDER BY
    order_month;

-- SEE TOP 5 SPENDING CUSTOMERS
SELECT 
    customer_name,
    SUM(total_revenue) AS total_revenue_generated
FROM sales_performance
GROUP BY customer_name, customer_id
ORDER BY SUM(total_revenue) DESC
LIMIT(5);

-- SEE REVENUE BY CUSTOMER SEGMENTS
SELECT 
    segment,
    SUM(total_revenue) AS total_revenue_generated
FROM sales_performance
GROUP BY segment
ORDER BY SUM(total_revenue) DESC;

-- SEE PRODUCT CATEGORY PERFORMANCE
SELECT
    category,
    SUM(total_revenue) AS total_revenue_generated
FROM sales_performance
GROUP BY category
ORDER BY SUM(total_revenue) DESC;

-- SEE REGION PERFORMANCE
SELECT 
    region,
    SUM(total_revenue) AS total_revenue_generated
FROM sales_performance
GROUP BY region
ORDER BY SUM(total_revenue) DESC;

-- SEE TOP 3 PERFORMING MONTHS
SELECT 
    order_month,
    SUM(total_revenue) AS total_revenue_generated
FROM sales_performance
GROUP BY order_month
ORDER BY SUM(total_revenue) DESC
LIMIT(3);

-- CHECK REVENUE TOTALS
SELECT 
    SUM(total_revenue)
FROM sales_performance;

SELECT 
    SUM(total_revenue)
FROM fact_sales;

SELECT 
    SUM(CAST(total_amount AS FLOAT))
FROM raw_orders;
"""


In [11]:
# Function to execute the SQL script
def execute_sql_script(script, connection_params):
    try:
        with psycopg2.connect(**connection_params) as conn:
            with conn.cursor() as cursor:
                cursor.execute(script)
                conn.commit()
                print("SQL script executed successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Execute the script
execute_sql_script(sql_script, DATABASE_URI)

SQL script executed successfully.
